In [1]:
import cv2
import torch
from ultralytics import YOLO

from common.yolo.yolo_results_aux import results_to_pose_list, pose_list_to_numpy, numpy_to_pose_list
from common.yolo.nms import nms_for_yolo_pose
from common.tracker.sort_tracker import SortTracker
from common.yolo.visualization import draw_bboxes_without_labels

In [2]:
# Load the YOLOv8n-Pose model
model = YOLO("/opt/models/yolo/yolov8n-pose.pt")

# Test whether the GPU device is available
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# Move the model to the device
model.to(DEVICE)

# Open the video file
cap = cv2.VideoCapture("/opt/videos/shopping_mall_01.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

In [ ]:
# Define the screen and window dimensions
screen_width = 1920  # Replace with actual screen width
screen_height = 1080  # Replace with actual screen height
window_width = 800  # Window width as specified in cv2.resize()
window_height = 600  # Window height as specified in cv2.resize()

# Create the SORT tracker
tracker = SortTracker(max_age=10, min_hits=1, iou_threshold=0.5)

# Set up the display window and position it at the bottom-right corner
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
cv2.moveWindow("Frame", screen_width - window_width, screen_height - window_height)

# Process the video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame, verbose=False)[0] # Get the first result

    # Use a list to store the detected boxes
    dets_data = []

    # Process the pose
    poses = results_to_pose_list(results)

    # Apply NMS to the poses
    poses = nms_for_yolo_pose(poses)

    # Convert the list of poses to a numpy array
    poses_array = pose_list_to_numpy(poses)

    # Perform object tracking
    tracked_objects = tracker.update(poses_array)

    # Draw the bboxes without labels
    frame = draw_bboxes_without_labels(frame, poses)

    # Display the frame
    frame = cv2.resize(frame, (window_width, window_height))
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 